In [1]:
#imports
import boto3
import pyodbc
import pandas as pd
import redshift_connector
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import os
import json
from functools import reduce

In [2]:
with open('aws.json') as json_file:
    cr = json.load(json_file)

s3creds = cr['data_sources']['s3']['creds']
redcreds = cr['data_sources']['redshift']['creds']

In [4]:
KEYPATH = "gcp.json"
CREDS = service_account.Credentials.from_service_account_file(KEYPATH)
client = bigquery.Client(credentials=CREDS, project=CREDS.project_id)


q1 = """
SELECT * FROM training_dataset.tournament
"""
q2 = """
SELECT * FROM training_dataset.tournament_winner
"""

view1 = client.query(q1)
view2 = client.query(q2)
tournament = view1.result().to_dataframe()
tournament_winner = view2.result().to_dataframe()


In [8]:
bo3 = tournament[tournament.best_of==3]

,ATP,Location,Tournament,tournament_date,Series,Court,Surface,Round,best_of,Comment,row_order
0,27,Lyon,Lyon Open,05-17-2021,ATP250,Outdoor,Clay,1st Round,3,Completed,14947
1,27,Lyon,Lyon Open,05-17-2021,ATP250,Outdoor,Clay,1st Round,3,Completed,14948
2,27,Lyon,Lyon Open,05-17-2021,ATP250,Outdoor,Clay,1st Round,3,Completed,14949
3,27,Lyon,Lyon Open,05-17-2021,ATP250,Outdoor,Clay,1st Round,3,Completed,14950
4,27,Lyon,Lyon Open,05-18-2021,ATP250,Outdoor,Clay,1st Round,3,Completed,14951
...,...,...,...,...,...,...,...,...,...,...,...
35918,63,St. Petersburg,St. Petersburg Open,10-24-2008,International,Indoor,Carpet,Quarterfinals,3,Completed,35009
35919,63,St. Petersburg,St. Petersburg Open,10-24-2008,International,Indoor,Carpet,Quarterfinals,3,Completed,35010
35920,63,St. Petersburg,St. Petersburg Open,10-25-2008,International,Indoor,Carpet,Semifinals,3,Completed,35011
35921,63,St. Petersburg,St. Petersburg Open,10-25-2008,International,Indoor,Carpet,Semifinals,3,Completed,35012


In [11]:
flags = ['ARG','AUS','UKR','ISR']
tournament_winner[tournament_winner['pl1_flag'].isin(flags)]

,ATP,pl1_flag,pl1_year_pro,pl1_weight,pl1_height,pl1_hand,row_order
44,1,ARG,2010,64,170,Right-Handed,149
45,4,ARG,2010,64,170,Right-Handed,1433
46,4,ARG,2010,64,170,Right-Handed,1482
47,4,ARG,2010,64,170,Right-Handed,1510
48,4,ARG,2010,64,170,Right-Handed,1657
...,...,...,...,...,...,...,...
32479,3,UKR,2005,88,188,Right-Handed,1212
32480,18,UKR,2005,88,188,Right-Handed,9130
32481,20,UKR,2005,88,188,Right-Handed,11359
32482,37,UKR,2005,88,188,Right-Handed,21875


In [12]:
merged = reduce(lambda left,right: pd.merge(left,right,on='row_order'),[tournament,tournament_winner])

In [16]:
stdCols=['row_order','Tournament','tournament_date','best_of','pl1_flag']
mergedStd = merged[stdCols]
mergedStd

,row_order,Tournament,tournament_date,best_of,pl1_flag
0,14947,Lyon Open,05-17-2021,3,GBR
1,14948,Lyon Open,05-17-2021,3,GBR
2,14949,Lyon Open,05-17-2021,3,JPN
3,14950,Lyon Open,05-17-2021,3,USA
4,14951,Lyon Open,05-18-2021,3,ITA
...,...,...,...,...,...
35918,35009,St. Petersburg Open,10-24-2008,3,ROU
35919,35010,St. Petersburg Open,10-24-2008,3,GBR
35920,35011,St. Petersburg Open,10-25-2008,3,KAZ
35921,35012,St. Petersburg Open,10-25-2008,3,GBR


In [50]:
mergedStd['monthEnd'] = pd.to_datetime(mergedStd['tournament_date']) + pd.offsets.MonthEnd()

In [54]:
mergedStd['monthBeg'] = pd.to_datetime(mergedStd['tournament_date']) + pd.offsets.MonthEnd() - pd.offsets.MonthBegin()

In [58]:
mergedStd['dayNextYear'] = (pd.to_datetime(mergedStd['tournament_date']) + pd.offsets.DateOffset(years=1)).dt.day_name()

In [61]:
uploadData = mergedStd
uploadData

,row_order,Tournament,tournament_date,best_of,pl1_flag,monthEnd,monthBeg,dayNextYear
0,14947,Lyon Open,05-17-2021,3,GBR,2021-05-31,2021-05-01,Tuesday
1,14948,Lyon Open,05-17-2021,3,GBR,2021-05-31,2021-05-01,Tuesday
2,14949,Lyon Open,05-17-2021,3,JPN,2021-05-31,2021-05-01,Tuesday
3,14950,Lyon Open,05-17-2021,3,USA,2021-05-31,2021-05-01,Tuesday
4,14951,Lyon Open,05-18-2021,3,ITA,2021-05-31,2021-05-01,Wednesday
...,...,...,...,...,...,...,...,...
35918,35009,St. Petersburg Open,10-24-2008,3,ROU,2008-10-31,2008-10-01,Saturday
35919,35010,St. Petersburg Open,10-24-2008,3,GBR,2008-10-31,2008-10-01,Saturday
35920,35011,St. Petersburg Open,10-25-2008,3,KAZ,2008-10-31,2008-10-01,Sunday
35921,35012,St. Petersburg Open,10-25-2008,3,GBR,2008-10-31,2008-10-01,Sunday


In [62]:
pandas_gbq.to_gbq(uploadData, destination_table = 'training_dataset.DA-16', project_id=CREDS.project_id, credentials = CREDS,if_exists='replace')